In [1]:
from tensorflow.examples.tutorials.mnist import input_data
from sklearn.svm import LinearSVC
import numpy as np
from linear_models import LinearOneVsAllClassifier
from mwu import runMWU
from helper import generate_data
from setup_mnist import *
from noise_functions_multiclass import gradientDescentMulti, gradientDescentNonConvex

Using TensorFlow backend.


In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
mnist_images = np.copy(mnist.train.images)
mnist_labels = np.argmax(mnist.train.labels, axis=1)

In [4]:
mnist_test_images = mnist.test.images
mnist_test_labels = np.argmax(mnist.test.labels, axis=1)

In [5]:
num_mnist_features = 784

In [ ]:
num_models = 1000
linear_models = []
zeroed_features_list = []
for i in xrange(num_models):
    if i % 10 == 0:
        print i
    sparse_mnist_images = np.copy(mnist_images)
    zeroed_features = np.random.randint(0, num_mnist_features, 588)
    zeroed_features_list.append(zeroed_features)
    sparse_mnist_images[:, zeroed_features] = 0.0
    
    model = LinearSVC(loss='hinge')
    model.fit(sparse_mnist_images, mnist_labels)
    
    linear_models.append(model)

In [ ]:
len(linear_models)

In [ ]:
for i, model in enumerate(linear_models):
    np.save('test/w_{}.npy'.format(i), model.weights)
    np.save('test/b_{}.npy'.format(i), model.bias)

In [6]:
linear_models = [LinearOneVsAllClassifier(10, np.load('test/w_{}.npy'.format(i)), np.load('test/b_{}.npy'.format(i)))
                 for i in range(900)]

In [ ]:
linear_models = [LinearOneVsAllClassifier(10, model.coef_, model.intercept_) for model in linear_models]

In [ ]:
accuracies = [model.evaluate(mnist_test_images, mnist_test_labels) for model in linear_models]
print "average model accuracy ", np.mean(accuracies)
print "min model accuracy ", np.min(accuracies)
print "max model accuracy ", np.max(accuracies)

In [ ]:
len(linear_models)

In [7]:
X_exp, Y_exp = generate_data(100, mnist_images, mnist_labels, linear_models)

In [8]:
np.save("X_exp.npy", X_exp)
np.save("Y_exp.npy", Y_exp)

In [10]:
accuracies = []
for lb, ub in [(0, 5), (5, 15), (15, 65), (65, 165), (165, 365)]:
    model_subset = linear_models[lb:ub]
    _, v, _, _, _ = runMWU(model_subset, 1, X_exp, Y_exp, 2.8, gradientDescentMulti, "test")
    accs = [model.evaluate(X_exp + v[0], Y_exp) for model in linear_models]
    print "Results "
    print ub - lb, np.mean(accs), np.min(accs), np.max(accs)
    accuracies.append(accs)


Running MWU for 1 Iterations with Epsilon 1.26863624118

Guaranteed to be within 2.53727248236 of the minimax value 

Iteration 0

Weights [0.2 0.2 0.2 0.2 0.2] Sum of Weights 1.0
Maximum (Average) Accuracy of Classifier 0.0
Cost (Before Noise) [0. 0. 0. 0. 0.]
Cost (After Noise), [1. 1. 1. 1. 1.]
Loss 1.0 Loss Per Action [0.2, 0.2, 0.2, 0.2, 0.2]
time spent 7.99998497963

finished running MWU 
Results 
5 0.8443555555555555 0.0 0.99

Running MWU for 1 Iterations with Epsilon 1.51742712939

Guaranteed to be within 3.03485425877 of the minimax value 

Iteration 0

Weights [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1] Sum of Weights 1.0
Maximum (Average) Accuracy of Classifier 0.0
Cost (Before Noise) [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Cost (After Noise), [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Loss 1.0 Loss Per Action [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
time spent 40.0824010372

finished running MWU 
Results 
10 0.5598222222222222 0.0 0.87

Running MWU for 1 Iterations with Epsilon 1.977

In [ ]:
noise = np.array(v)

In [ ]:
np.save('noise.npy', noise)

In [ ]:
noise = np.load('noise.npy')[0]

In [ ]:
model_dir = "deep_networks"
models = [conv_net(False, 2, 200, model_dir + "/conv0"), conv_net(True, 2, 200, model_dir + "/conv1"), 
          conv_net(True, 4, 64, model_dir + "/conv2"), multilayer(4, 128, model_dir + "/mlp0"),
          multilayer(2, 256, model_dir + "/mlp1")]

In [ ]:
X = X_exp.reshape(-1, 28, 28, 1)
V = noise[0].reshape(-1, 28, 28, 1)
Y = np.array([(np.arange(10) == l).astype(np.float32) for l in Y_exp])
[model.evaluate(X + V, Y) for model in models]

In [ ]:
[model.evaluate(X, Y)[1] for model in models]

In [ ]:
t.shape

In [ ]:
model